In [11]:
#!pip install requests beautifulsoup4

In [ ]:
import requests
import time
import os
import re
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.common.by import By
from datetime import datetime

In [ ]:

# Set up Edge options
edge_options = EdgeOptions()
edge_options.add_argument("headless")  # Remove this line if you want to see the browser

# Path to Edge WebDriver
service = EdgeService(r"C:\Users\alons\Downloads\Installers\edgedriver_win64\msedgedriver.exe")

# Start the WebDriver
driver = webdriver.Edge(service=service, options=edge_options)

# Base URL template
base_url = (
    "https://auctions.royaltyexchange.com/orderbook/past-deals/"
    "?filter{state.in}=filled&filter{state.in}=pending&filter{state.in}=closed"
    "&sort[]=-deal_date&sort[]=-published_date&page_size=50"
)

# Load page 1 to detect number of pages
driver.get(f"{base_url}&page=1")
time.sleep(2)
buttons = driver.find_elements(By.CSS_SELECTOR, "ul.MuiPagination-ul > li button")
page_numbers = [int(btn.text) for btn in buttons if btn.text.isdigit()]
last_page = max(page_numbers) if page_numbers else 1

all_html = []
for page_num in range(1, last_page + 1):
    print(f"Fetching page {page_num}…")
    url = f"{base_url}&page={page_num}"
    driver.get(url)
    time.sleep(2)
    all_html.append(driver.page_source)

driver.quit()

output_path = os.path.join(os.getcwd(), "royaltyexchange_pages.txt")
with open(output_path, "w", encoding="utf-8") as f:
    f.write("\n\n" + ("="*100) + "\n\n".join(all_html))

print(f"✅ Done! File saved to: {output_path}")


Fetching page 1…
Fetching page 2…
Fetching page 3…
Fetching page 4…
Fetching page 5…
Fetching page 6…
Fetching page 7…
Fetching page 8…
Fetching page 9…
Fetching page 10…
Fetching page 11…
Fetching page 12…
Fetching page 13…
Fetching page 14…
Fetching page 15…
Fetching page 16…
Fetching page 17…
Fetching page 18…
Fetching page 19…
Fetching page 20…
Fetching page 21…
Fetching page 22…
Fetching page 23…
Fetching page 24…
Fetching page 25…
Fetching page 26…
Fetching page 27…
Fetching page 28…
Fetching page 29…
Fetching page 30…
Fetching page 31…
Fetching page 32…
Fetching page 33…
Fetching page 34…
Fetching page 35…
Fetching page 36…
Fetching page 37…
Fetching page 38…
Fetching page 39…
Fetching page 40…
Fetching page 41…
Fetching page 42…
Fetching page 43…
Fetching page 44…
✅ Done! File saved to: c:\Users\alons\OneDrive - Cornell University\Cornell University\Spring 2025\CFEM Research Assistant\Parser\royalty_scraper\royaltyexchange_pages.txt


In [14]:

# Load content from file
with open("royaltyexchange_pages.txt", "r", encoding="utf-8") as f:
    new_content = f.read()

# Extract all numbers that follow "ID:"
new_ids = re.findall(r'ID:\s*(\d+)', new_content)

# Create a DataFrame
df_new_ids = pd.DataFrame(new_ids, columns=['Listing ID'])

# Generate date string
today_str = datetime.today().strftime('%Y-%m-%d')

# Save to CSV with date
filename = f"new_extracted_listing_ids_{today_str}.csv"
df_new_ids.to_csv(filename, index=False)

print(f"Saved to {filename}")

Saved to new_extracted_listing_ids_2025-04-22.csv


### Filter & Update Existing Universe
This code ensures that only truly new listings get added to your master universe.

In [ ]:
# load existing universe of asset ids
master_path = "asset_ids_total.csv"
master_df   = pd.read_csv(master_path, dtype=str)
master_ids  = set(master_df["asset_id"])

# load new‑extracted file
today       = datetime.today().strftime('%Y-%m-%d')
new_path    = f"new_extracted_listing_ids_{today}.csv"
new_df      = pd.read_csv(new_path, dtype=str)

# if your CSV uses a different header, e.g. "Listing ID", rename it:
if "Listing ID" in new_df.columns:
    new_df = new_df.rename(columns={"Listing ID": "asset_id"})

new_ids = set(new_df["asset_id"])

# filter down to only the truly new ones. no duplicates will be added to the universe
unique_ids = new_ids - master_ids
unique_df = new_df[new_df["asset_id"].isin(unique_ids)]
unique_path = f"unique_new_listing_ids_{today}.csv"
unique_df.to_csv(unique_path, index=False)
print(f"{len(unique_df)} new listings → {unique_path}")

# update universe list
if not unique_df.empty:
    updated = pd.concat([master_df, unique_df], ignore_index=True)
    updated.to_csv(master_path, index=False)
    print(f"Appended {len(unique_df)} to {master_path}")


986 new listings → unique_new_listing_ids_2025-04-22.csv
Appended 986 to asset_ids_total.csv


: 